文本的embedding是一种将文本转换成数值形式的技术，这使得计算机能够理解和处理自然语言。在机器学习和自然语言处理中，这种技术非常关键，它能够捕捉到单词、短语或整段文本的语义信息。
具体来说，文本embedding通常是通过将单词或短语映射到一个高维空间（通常是数百维）的向量来实现的。这些向量是通过训练算法（如word2vec、GloVe或BERT等）在大量文本数据上学习得到的。每个向量的维度代表了一种潜在的语义特征，向量本身则捕捉到了词汇的语义内容和使用上下文。
这样的向量表示形式可以使机器学习模型更有效地处理语言数据，因为它们将文本转换为模型可以操作的数值形式。通过这种方式，模型可以执行各种任务，如文本分类、情感分析、机器翻译等。

接下来我们以《劳动法》为例，使用LLM将每条法条表示为一个embedding，然后进行基于这些embedding的探索

In [1]:
import re

with open("laodongfa.txt", "r") as fp:
    text = fp.read().replace("\u3000", " ")
pattern = r"(  第.*?条.*?)(?=  第.*?条|第.*?章|$)"
matches = re.findall(pattern, text, re.DOTALL)

rules = []
for index, match in enumerate(matches):
    rules.append(match.strip())
# rules中每个元素是一条法条的文本

In [2]:
# 我们使用qwen:4b模型来为文本计算embedding
# ollama是一个本地调用LLM的库
from langchain_community.embeddings.ollama import OllamaEmbeddings

embedding = OllamaEmbeddings(model="qwen:4b")
print(embedding.embed_documents(["你好啊"]))


[[-0.16074761748313904, 18.353322982788086, 1.8791629076004028, 1.6699440479278564, 2.4595043659210205, 7.4938645362854, 3.016930341720581, 4.259737968444824, 1.4110828638076782, 2.4469716548919678, 1.472164273262024, 4.73881196975708, -5.362461566925049, -0.2840634286403656, -6.306908130645752, 2.983499050140381, 1.648193359375, 0.17265577614307404, -1.1411365270614624, -3.3243751525878906, 6.687701225280762, -1.7878261804580688, -1.352903127670288, 1.034665584564209, -28.857215881347656, 4.388082504272461, 0.31274041533470154, 4.096693992614746, -4.637551307678223, 4.106581687927246, 1.2953070402145386, 2.906195640563965, 2.0816569328308105, -3.2777698040008545, 3.0629050731658936, 1.3998079299926758, 1.9177955389022827, -5.15391731262207, 5.018749713897705, -6.577813625335693, -0.3670200705528259, 0.3253405690193176, 1.9581738710403442, -0.20952072739601135, 2.658404588699341, -0.5215789079666138, 1.8315024375915527, -0.24823510646820068, -0.7711488008499146, 0.3576226830482483, 2.5

In [3]:
# chroma是一个向量数据库
from langchain_community.vectorstores.chroma import Chroma

# 将所有法条文本，经过上面定义的embedding方法计算成向量，存入向量数据库，并且存到persist_directory目录中
vectorstore = Chroma.from_texts(
    texts=rules,
    embedding=embedding, 
    persist_directory="./laodongfa.emb"
)

接下来我们进行一些搜索尝试，给一段文本，通过向量近似，获取最相关的5个法条。

In [5]:
vectorstore.similarity_search("未成年工定义是什么", k=5)

[Document(page_content='第一条 为了保护劳动者的合法权益，调整劳动关系，建立和维护适应社会主义市场经济的劳动制度，促进经济发展和社会进步，根据宪法，制定本法。'),
 Document(page_content='第五十八条 国家对女职工和未成年工实行特殊劳动保护。\n  未成年工是指年满十六周岁未满十八周岁的劳动者。'),
 Document(page_content='第三十六条 国家实行劳动者每日工作时间不超过八小时、平均每周工作时间不超过四十四小时的工时制度。'),
 Document(page_content='第四十五条 国家实行带薪年休假制度。\n  劳动者连续工作一年以上的，享受带薪年休假。具体办法由国务院规定。'),
 Document(page_content='第十五条 禁止用人单位招用未满十六周岁的未成年人。文艺、体育和特种工艺单位招用未满十六周岁的未成年人，必须依照国家有关规定，履行审批手续，并保障其接受义务教育的权利。')]

In [6]:
vectorstore.similarity_search("每日工作时间不能超过多少？", k=5)

[Document(page_content='第一条 为了保护劳动者的合法权益，调整劳动关系，建立和维护适应社会主义市场经济的劳动制度，促进经济发展和社会进步，根据宪法，制定本法。'),
 Document(page_content='第五十六条 劳动者在劳动过程中必须严格遵守安全操作规程。\n  劳动者对用人单位管理人员违章指挥、强令冒险作业，有权拒绝执行；对危害生命安全和身体健康的行为，有权提出批评、检举和控告。'),
 Document(page_content='第四十五条 国家实行带薪年休假制度。\n  劳动者连续工作一年以上的，享受带薪年休假。具体办法由国务院规定。'),
 Document(page_content='第十条 国家通过促进经济和社会发展，创造就业条件，扩大就业机会。国家鼓励企业、事业组织、社会团体在法律、行政法规规定的范围内兴办产业或者拓展经营，增加就业。国家支持劳动者自愿组织起来就业和从事个体经营实现就业。'),
 Document(page_content='第三十六条 国家实行劳动者每日工作时间不超过八小时、平均每周工作时间不超过四十四小时的工时制度。')]

In [7]:
vectorstore.similarity_search("公司暴力裁员，我该怎么办？", k=5)

[Document(page_content='第一条 为了保护劳动者的合法权益，调整劳动关系，建立和维护适应社会主义市场经济的劳动制度，促进经济发展和社会进步，根据宪法，制定本法。'),
 Document(page_content='第七条 劳动者有权依法参加和组织工会。工会代表和维护劳动者的合法权益，依法独立自主地开展活动。'),
 Document(page_content='第一百零五条 违反本法规定侵害劳动者合法权益，其他法律、法规已规定处罚的，依照该法律、行政法规的规定处罚。'),
 Document(page_content='第十条 国家通过促进经济和社会发展，创造就业条件，扩大就业机会。国家鼓励企业、事业组织、社会团体在法律、行政法规规定的范围内兴办产业或者拓展经营，增加就业。国家支持劳动者自愿组织起来就业和从事个体经营实现就业。'),
 Document(page_content='第三十四条 集体合同签订后应当报送劳动行政部门；劳动行政部门自收到集体合同文本之日起十五日内未提出异议的，集体合同即行生效。')]

可以看到，最后一个case搜索出来的内容不是很相关，目前的搜索方案是基于语义相似，并不是完善的QA系统，所以有的情况搜到的内容也不是非常好。

回顾一下，本节课程我们学习了如何使用ollama来对文本进行向量化，以及使用chroma向量库来进行向量近似检索。